In [35]:
file_path = r'/home/jovyan/genetic_algo/genetic.py'

# Read the contents of the file
with open(file_path, 'rb') as f:
    file_contents = f.read()

# Write the contents of the file to a new file
with open('genetic.py', 'wb') as src:
    src.write(file_contents)
    
print(src)

<_io.BufferedWriter name='genetic_2.py'>


In [36]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Apr 25 17:28:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:65:00.0 Off |                    0 |
|  0%   39C    P0    79W / 300W |  43837MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [37]:
from genetic import geneticMaterial

from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from statistics import mean
import random
# loads top 5000 words from the IMDB Dataset
#x_train and x_test are lists of sequences of integers representing the words in the movie reviews.
#y_train and y_test are lists of binary labels, where 0 represents a negative review and 1 represents a positive review. The labels are associated with the corresponding reviews in x_train and x_test.
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)

In [38]:
embedding_dimension = 50
top_words = 5000

In [39]:
# converts each length of review to maximum length, by padding or truncating at the end of the sequence
def get_data(maxlen):
  x = sequence.pad_sequences(x_train, maxlen=maxlen,padding='post',truncating='post')
  return x

In [40]:
# Initialisation - generating the population for first generation
population = [geneticMaterial() for i in range(geneticMaterial.num_population)]
for each in population:
  print(each.phenoType)

[823, 41, 4, 13]
[870, 346, 0, 127]
[416, 475, 13, 314]
[646, 362, 8, 216]
[638, 117, 11, 150]
[341, 120, 3, 503]
[1018, 116, 14, 225]
[829, 235, 4, 119]
[375, 476, 0, 441]
[6, 207, 13, 272]
[839, 94, 0, 3]
[452, 123, 5, 32]
[123, 458, 3, 274]
[803, 237, 0, 368]
[56, 413, 12, 359]
[114, 241, 4, 431]
[358, 153, 4, 79]
[830, 151, 14, 483]
[704, 252, 5, 268]
[280, 139, 4, 271]


In [41]:
# saved phenotype from last iteration
phenotypes= [[823, 41, 4, 13],
[870, 346, 0, 127],
[416, 475, 13, 314],
[646, 362, 8, 216],
[638, 117, 11, 150],
[341, 120, 3, 503],
[1018, 116, 14, 225],
[829, 235, 4, 119],
[375, 476, 0, 441],
[6, 207, 13, 272],
[839, 94, 0, 3],
[452, 123, 5, 32],
[123, 458, 3, 274],
[803, 237, 0, 368],
[56, 413, 12, 359],
[114, 241, 4, 431],
[358, 153, 4, 79],
[830, 151, 14, 483],
[704, 252, 5, 268],
[280, 139, 4, 271]]
for index,individual in enumerate(population):
  individual.phenoType = phenotypes[index]
  individual.convert_to_genotype()

In [42]:
# It represents the phenotype and the genotype space
for each in population:
  print(each.phenoType)
  print(each.gene)

[823, 41, 4, 13]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
[870, 346, 0, 127]
[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[416, 475, 13, 314]
[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0]
[646, 362, 8, 216]
[1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
[638, 117, 11, 150]
[1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
[341, 120, 3, 503]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[1018, 116, 14, 225]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1]
[829, 235, 4, 119]
[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]
[375, 476, 0, 441]
[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [43]:
# LSTM Network with hyperparameter values opted from the each individualin the pheotype space 
def create_model(individual):
    model = Sequential()
    # The embedding layer with length of each sequence taken from the individual of our phenotype space
    # minimum input length kept to 1
    model.add(Embedding(top_words, embedding_dimension,input_length=max(1,individual.phenoType[3]),mask_zero=True))
   
    # LSTM layer, hidden units value taken from the individual of our phenotype space
    # minimum input length kept to 1
    model.add(LSTM(units = max(1,individual.phenoType[1])))
    
    # Dropout layer with value 0.2 to reduce over-fitting
    model.add(Dropout(0.2))
    
    # Dense layer with a single output neuron,
    # and a sigmoid activation function to produce a probability score.
    model.add(Dense(1,activation='sigmoid'))
    
    # Learning rate of the optimiser is taken from the individual of our phenotype space
    # minimum value capped to 0.0005;
    #since learning rate value range to 1023, divided by 2000 to keep it within 0.511
    model.compile(optimizer=Adam(learning_rate=max(0.0005,individual.phenoType[0]/2000)),loss='binary_crossentropy',metrics=['accuracy'])
   
    # epoch value taken from individual from our phenotype space
    epochs = individual.phenoType[2]
    
    # To keep the minimum length of review to 1.
    data_array = get_data(max(1,individual.phenoType[3]))
    return model, epochs, data_array

In [44]:
# Returns list of probability for each individual in the population
# Fitness value is raised to the fourth power, to increase the difference between higher fitness value to lower fitness value
# making it easier to select the best individuals for the next generation
def get_probability():
    powered_fitness = [pow(each.fitness,4) for each in population]
    total = sum(powered_fitness)
    probability = [float(i/total) for i in powered_fitness]
    print("\nprobability array: ", probability)
    return probability


In [ ]:
from timeit import default_timer as timer
start = timer()

# running this loop for max_generations
for generation_ in range(geneticMaterial.max_generations): # replace whatever inside range with geneticMaterial.max_generations
    for each in population:
        model, epochs,data_array = create_model(each)
        print("PHENOTYPE ",each.phenoType)
        print("learning rate = ",each.phenoType[0]/2000)
        history = model.fit(data_array,y_train,epochs=max(1,epochs),batch_size=100,validation_split=0.2,verbose=0)
        # The fitness value is being set based on the last recorded validation accuracy value
        #stored in a dictionary called "history.history". 
        each.set_fitness(history.history['val_accuracy'][-1])
        print("validation accuracy last epochs ", history.history['val_accuracy'][-1])

    probability = get_probability()
    new_population = []

    # Retain best performing model to next generation
    best_model_in_a_generation = probability.index(max(probability))
    print("Best model, generation {}, is {}".format(generation_+1,population[best_model_in_a_generation].phenoType))
    new_population.append(population[best_model_in_a_generation])
    
    for i in range(1,len(population)):
        # select first parent for crossover
        point = random.random()
        accumulated_probability = probability[0]
        parent_1 = None
        parent_2 = None
        parent_1_index = None
        for j in range(len(population)):
            if point < accumulated_probability:
                parent_1 = population[j]
                parent_1_index = j
                break
            else:
                accumulated_probability += probability[j+1]

        second_parented_selected = False
        # select the second parent
        while not second_parented_selected:
            point = random.random()
            accumulated_probability = probability[0]
            for j in range(len(population)):
                if point < accumulated_probability and population[j].phenoType != population[parent_1_index].phenoType:
                    parent_2 = population[j]
                    second_parented_selected = True
                    break
                else:
                    accumulated_probability += probability[j]
        new_individual = geneticMaterial()
        # select crossover point
        crossover_point = sum(geneticMaterial.gene_num_bits[:len(geneticMaterial.gene_num_bits)//random.randint(2,len(geneticMaterial.gene_num_bits)-1)])
        new_individual.crossover(parent_1, parent_2, crossover_point=crossover_point)

        # muutate if random number less than mutation probability
        if random.random() < geneticMaterial.mutation_probability:
            new_individual.mutate()

        new_population.append(new_individual)
    population = new_population
end = timer()
print("time taken = ",end-start)
print(population)




GENERATION 1
PHENOTYPE  [823, 41, 4, 13]
learning rate =  0.4115
validation accuracy last epochs  0.4959999918937683
PHENOTYPE  [870, 346, 0, 127]
learning rate =  0.435
validation accuracy last epochs  0.5062000155448914
PHENOTYPE  [416, 475, 13, 314]
learning rate =  0.208
validation accuracy last epochs  0.49380001425743103
PHENOTYPE  [646, 362, 8, 216]
learning rate =  0.323
validation accuracy last epochs  0.5062000155448914
PHENOTYPE  [638, 117, 11, 150]
learning rate =  0.319
validation accuracy last epochs  0.5090000033378601
PHENOTYPE  [341, 120, 3, 503]
learning rate =  0.1705
validation accuracy last epochs  0.5198000073432922
PHENOTYPE  [1018, 116, 14, 225]
learning rate =  0.509
validation accuracy last epochs  0.5023999810218811
PHENOTYPE  [829, 235, 4, 119]
learning rate =  0.4145
validation accuracy last epochs  0.5098000168800354
PHENOTYPE  [375, 476, 0, 441]
learning rate =  0.1875
validation accuracy last epochs  0.49380001425743103
PHENOTYPE  [6, 207, 13, 272]
lea

In [ ]:
from matplotlib import pyplot as plt
y_axis = [0.7817999720573425, 0.8745999932289124,0.8820000290870667,0.8736000061035156,0.8751999735832214,0.8804000020027161,0.876800000667572,0.8776000142097473, 0.8858000040054321, 0.8880000114440918, 0.8835999965667725, 0.8840000033378601, 0.8809999823570251, 0.8907999992370605, 0.8646000027656555, 0.8866000175476074, 0.8870000243186951, 0.8835999965667725, 0.8817999958992004, 0.876800000667572, 0.8772000074386597, 0.8840000033378601]
x_axis = [i for i in range(1,23)]
plt.plot(x_axis,y_axis,'-o')
plt.xlabel('Generation')
plt.ylabel('Mean Validation accuracy')